# Лекция Python 1
## Один из реалистичных примеров ООП
### Объектно-ориентированное программирование и наследование: отношения "является"

In [34]:
from __future__ import print_function

class Employee:
    def __init__(self, name, salary=0):
        self.name = name
        self.salary = salary
    
    def giveRaise(self, percent):
        self.salary = self.salary + (self.salary * percent)

    def work(self):
        print(self.name, "does stuff") # типо что-то делает

    def __repr__(self):
        return "<Employee: name=%s, salary=%s>" % (self.name, self.salary)

class Chef(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 50000)

    def work(self):
        print(self.name, "makes food") # готовит еду

class Server(Employee):
    def __init__(self, name):
        Employee.__init__(self, name, 40000)
    
    def work(self):
        print(self.name, "interfaces with customer") # Взаимодействует с клиентом

class PizzaRobot(Chef):
    def __init__(self, name):
        Chef.__init__(self, name)   

    def work(self):
        print(self.name, "makes pizza") # готовит пиццу

if __name__ == "__main__" :
    bob = PizzaRobot ('bob')            # Создать робота по имени bob
    print(bob)                          # Выполняется унаследованный метод__repr__
    bob.work()                          # Выполняется действие, специфичное для типа
    bob.giveRaise (0.20)                # Повысить зарплату роботу bob на 20%
    print(bob); print()
    
for klass in Employee, Chef, Server, PizzaRobot: 
    obj = klass(klass.__name__)
    obj.work()

<Employee: name=bob, salary=50000>
bob makes pizza
<Employee: name=bob, salary=60000.0>

Employee does stuff
Chef makes food
Server interfaces with customer
PizzaRobot makes pizza


### Объектно-ориентированное программирование и наследование: отношения "имеет"

In [35]:
# Файл pizzashop.ру (Python 2.Х + З.Х)
from __future__ import print_function 

class Customer:
    def __init__(self, name):
        self.name = name
    
    def order(self, server):
        print(self.name, "orders from", server) # заказы от
    
    def pay(self, server):
        print(self.name, "pays for item to", server) # плата за единицу
        

class Oven:
    def bake(self): 
        print("oven bakes") # духовой шкаф выпекает

class StoneOven(Oven):
    def bake(self): 
        print("oven bakes on stone raft") # духовой шкаф выпекает на каменной подложке


class PizzaShop:
    def __init__(self) :
        self.server = Server('Pat') # Внедрить другие объекты
        self.chef = PizzaRobot('Bob')  # Робот по имени bob
        self.oven = StoneOven(Oven)

    def order(self, name):
        customer = Customer(name) # Активизировать другие объекты
        customer.order(self.server) # Заказы клиента, принятые официантом
        self.chef.work() 
        self.oven.bake() 
        customer.pay(self.server)

if __name__ == "__main__": 
    scene = PizzaShop() # Создать составной объект
    scene.order('Homer') # Эмулировать заказ клиента Homer
    print('...')
    scene.order('Shaggy') # Эмулировать заказ клиента Shaggy

Homer orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Homer pays for item to <Employee: name=Pat, salary=40000>
...
Shaggy orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
Shaggy pays for item to <Employee: name=Pat, salary=40000>


### Обработчики потоков данных

В качестве примера композиции, который может оказаться чуть более осязаемым, чем роботы по приготовлению пиццы, напишем обобщенную функцию обработчика потоков данных:

In [ ]:
def processor(reader, converter, writer): 
    while True:
        data = reader.read() 
        if not data: 
            break 
        data = converter(data) 
        writer.write(data)


Вместо применения простой функции мы можем написать код класса, который для выполнения своей работы использует композицию, чтобы обеспечить большую структурированность и поддержку наследования. 


In [3]:
class Processor:
    def __init__(self, reader, writer):
        self.reader = reader 
        self.writer = writer

    def process(self): 
        while True:
            data = self.reader.readline() 
            if not data: 
                break
            data = self.converter(data) 
            self.writer.write(data)

    def converter(self, data):
        assert False,  'converter must be defined' # Или сгенерировать исключение

In [5]:
class Uppercase(Processor):
    def converter(self, data):
        return data.upper()


if __name__ == '__main__' : 
    import sys
    obj = Uppercase(open('./trispam.txt'), sys.stdout) 
    obj.process()

SPAM
SPAM
SPAM!

In [6]:
class HTMLize:
    def write(self, line):
        print('<q1>%s</q1>' % line.rstrip())

Uppercase(open('trispam.txt'), HTMLize()).process()

<q1>SPAM</q1>
<q1>SPAM</q1>
<q1>SPAM!</q1>


In [40]:
import shelve

object = PizzaShop()
dbase = shelve.open('test2.txt') 
dbase ['key'] = object  # Сохранить под ключом

import shelve
dbase = shelve.open('test2.txt')
object = dbase['key']  
print(object)

In [43]:
shop = PizzaShop()
shop.server
shop.chef

import pickle
pickle.dump(shop, open('shopfile.txt', 'wb'))

In [44]:
import pickle
obj = pickle.load(open('shopfile.txt', 'rb'))
obj.server
obj.chef
obj.order('LSP')

LSP orders from <Employee: name=Pat, salary=40000>
Bob makes pizza
oven bakes
LSP pays for item to <Employee: name=Pat, salary=40000>


## Паттерн декоратор

Декоратор – это структурный паттерн. Цель которого – предоставление новых функциональных возможностей классам и объектам во время выполнения кода.

Чаще всего декоратор представляет собой абстрактный класс, принимающий в конструкторе объект, функциональность которого мы хотим расширить. Но в Python есть и встроенный механизм декораторов, который можно использовать.

**Случаи использования**

Необходимость назначить дополнительные обязанности объектам во время выполнения, не ломая код, который использует эти объекты;
По каким-то причинам невозможно расширить «цепочку обязанностей» объекта через наследование.  


**Пример кода**

Используя декораторы, вы можете обернуть объекты несколько раз, поскольку и цель, и декораторы реализуют один и тот же интерфейс.

Получаемый объект будет обладать объединенной и сложенной функциональностью всех декораторов.

In [1]:
from abc import ABC, abstractmethod

class Component(ABC):
  @abstractmethod
  def operation(self):
      pass
  
  
class ConcreteComponent(Component):
  def operation(self):
      return "ConcreteComponent"


class Decorator(Component):
  def __init__(self, component):
      self.component = component
  @abstractmethod
  def operation(self):
      pass


class ConcreteDecoratorA(Decorator):
  def operation(self):
      return f"ConcreteDecoratorA({self.component.operation()})"
  
class ConcreteDecoratorB(Decorator):
  def operation(self):
      return f"ConcreteDecoratorB({self.component.operation()})"
  

if __name__ == "__main__":
  concreteComponent = ConcreteComponent()
  print(concreteComponent.operation())
  decoratorA = ConcreteDecoratorA(concreteComponent)
  decoratorB = ConcreteDecoratorB(concreteComponent)
  decorator = ConcreteDecoratorB(concreteComponent)
  concreteComponent.operation()
  
  print(decoratorB.operation())

ConcreteComponent
ConcreteDecoratorB(ConcreteDecoratorA(ConcreteComponent))


In [ ]:

@function2
def function():
    pass

In [2]:
import sys
def memoize(f):
  cache = dict()
  def wrapper(x):
      if x not in cache:
          cache[x] = f(x)
      return cache[x]
  return wrapper

@memoize
def fib(n):
  if n <= 1:
      return n
  else:
      return fib(n - 1) + fib(n - 2)
if __name__ == "__main__":
  sys.setrecursionlimit(2000)
  print(fib(750))

2461757021582324272166248155313036893697139996697461509576233211000055607912198979704988704446425834042795269603588522245550271050495783935904220352228801000
